# MissOh DataLoader

### AnotherMissOh Visual Structure
- json_data['file_name'] : 'AnotherMissOh01.mp4'
- json_data['visual_results']
- json_data['visual_results'][0].keys() : dict_keys(['start_time', 'end_time', 'vid', 'image_info'])
- {
'start_time': '00:02:51;16', 
'end_time': '00:02:54;15', 
'vid': 'AnotherMissOh01_001_0078', 
'image_info': ...}
- json_data['visual_results'][0]['image_info']
- [{'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004295', 
'place': 'none', 
'persons': [
{'person_id': 'Haeyoung1', 
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 845, 'max_y': 443}, 
'full_rect': {'min_x': 278, 'min_y': 2, 'max_x': 1025, 'max_y': 769}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}], 
'objects': []}, 
- {'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004311', 
'place': '', 
'persons': [{
'person_id':'Haeyoung1',
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 831, 'max_y': 411}, 
'full_rect': {'min_x': 270, 'min_y': 0, 'max_x': 1025, 'max_y': 768}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}],
'objects': []},]

In [1]:
# !apt-get install graphviz xdg-utils

In [2]:
import sys, os
sys.path.append("../") # go to parent dir

In [3]:
import os
from torch.utils.data import Dataset, DataLoader
import cv2
import pickle
import numpy as np
import glob
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import json
import argparse
import matplotlib.pyplot as plt

from Yolo_v2_pytorch.src.utils import *
from graphviz import Digraph, Graph

In [4]:
def is_not_blank(s):
    return bool(s and s.strip())

In [5]:
MissOh_CLASSES = ['person']
print(MissOh_CLASSES[0])
global colors
colors = pickle.load(open("../Yolo_v2_pytorch/src/pallete", "rb"))
print(colors[0])

person
(39, 129, 113)


In [6]:
import sys, os
sys.path.append("../") # go to parent dir

In [7]:
import os
import glob
import argparse
import pickle
import cv2
import numpy as np
from Yolo_v2_pytorch.src.utils import *
import torch.nn.functional as F
from torch.utils.data import DataLoader
from Yolo_v2_pytorch.src.yolo_net import Yolo
from Yolo_v2_pytorch.src.anotherMissOh_dataset import AnotherMissOh, Splits, SortFullRect, PersonCLS,PBeHavCLS, FaceCLS, ObjectCLS, P2ORelCLS
from torchvision.transforms import Compose, Resize, ToTensor
from PIL import Image
import matplotlib.pyplot as plt
import time

from lib.place_model import place_model, label_mapping, accuracy, label_remapping, place_buffer
from lib.person_model import person_model
from lib.behavior_model import behavior_model
from lib.pytorch_misc import optimistic_restore, de_chunkize, clip_grad_norm, flatten
from lib.focal_loss import FocalLossWithOneHot, FocalLossWithOutOneHot, CELossWithOutOneHot
from lib.face_model import face_model
from lib.object_model import object_model
from lib.relation_model import relation_model
from lib.emotion_model import emotion_model, crop_face_emotion, EmoCLS

num_persons = len(PersonCLS)
num_behaviors = len(PBeHavCLS)
num_faces = len(FaceCLS)
num_objects = len(ObjectCLS)
num_relations = len(P2ORelCLS)
num_emos = len(EmoCLS)

def get_args():
    parser = argparse.ArgumentParser(
        "You Only Look Once: Unified, Real-Time Object Detection")
    parser.add_argument("--image_size",
                        type=int, default=448,
                        help="The common width and height for all images")
    parser.add_argument("--batch_size", type=int, default=1,
                        help="The number of images per batch")
    parser.add_argument("--conf_threshold",
                        type=float, default=0.35)
    parser.add_argument("--nms_threshold",
                        type=float, default=0.5)
    parser.add_argument("--pre_trained_model_type",
                        type=str, choices=["model", "params"],
                        default="model")
    parser.add_argument("--data_path_test",
                        type=str,
                        default="./Yolo_v2_pytorch/missoh_test/",
                        help="the root folder of dataset")

    parser.add_argument("--saved_path", type=str,
                        default="./checkpoint/refined_models")

    parser.add_argument("--img_path", type=str,
                        default="./data/AnotherMissOh/AnotherMissOh_images_ver3.2/")
    parser.add_argument("--json_path", type=str,
                        default="./data/AnotherMissOh/AnotherMissOh_Visual_ver3.2/")
    parser.add_argument("-model", dest='model', type=str, default="baseline")
    parser.add_argument("-display", dest='display', action='store_true')
    parser.add_argument("-emo_net_ch", dest='emo_net_ch',type=int, default=64)
    args = parser.parse_args([])
    return args

# get args.
opt = get_args()
print(opt)

Namespace(batch_size=1, conf_threshold=0.35, data_path_test='./Yolo_v2_pytorch/missoh_test/', display=False, emo_net_ch=64, image_size=448, img_path='./data/AnotherMissOh/AnotherMissOh_images_ver3.2/', json_path='./data/AnotherMissOh/AnotherMissOh_Visual_ver3.2/', model='baseline', nms_threshold=0.5, pre_trained_model_type='model', saved_path='./checkpoint/refined_models')


In [8]:
opt.img_path = "../data/AnotherMissOh/AnotherMissOh_images_ver3.2/"
opt.json_path = "../data/AnotherMissOh/AnotherMissOh_Visual_ver3.2/"
opt.saved_path = "../checkpoint/refined_models"
opt.display = True

In [9]:
tform = [
    Resize((448, 448)),  # should match to Yolo_V2
    ToTensor(),
    # Normalize(# should match to Yolo_V2
    #mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]
transf = Compose(tform)

# splits the episodes int train, val, test
train, val, test = Splits(num_episodes=18)

# load datasets
train_set = AnotherMissOh(train, opt.img_path, opt.json_path, False)
val_set = AnotherMissOh(val, opt.img_path, opt.json_path, False)
test_set = AnotherMissOh(test, opt.img_path, opt.json_path, False)

# model path
model_path = "{}/anotherMissOh_{}.pth".format(
    opt.saved_path,opt.model)

In [10]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.manual_seed(123)
    device = torch.cuda.current_device()
else:
    torch.manual_seed(123)

# set test loader params
test_params = {"batch_size": opt.batch_size,
               "shuffle": False,
               "drop_last": False,
               "collate_fn": custom_collate_fn}

# set test loader
test_loader = DataLoader(test_set, **test_params)

# ---------------(1) load refined models --------------------
# get the trained models from
# https://drive.google.com/drive/folders/1WXzP8nfXU4l0cNOtSPX9O1qxYH2m6LIp
# emotion model
if True:
    model_emo = emotion_model(opt.emo_net_ch, num_persons, device)
    trained_emotion = '../checkpoint/refined_models' + os.sep + "{}".format(
    'anotherMissOh_only_params_emotion_integration.pth')
    model_emo.load_state_dict(torch.load(trained_emotion))
    print("loaded with {}".format(trained_emotion))
model_emo.cuda(device)
model_emo.eval()

True
loaded with ../checkpoint/refined_models/anotherMissOh_only_params_emotion_integration.pth


emotion_model(
  (yolo_net): YoloD(
    (stage1_conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (stage1_conv2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (stage1_conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, in

In [11]:
# load the color map for detection results
colors = pickle.load(open("../Yolo_v2_pytorch/src/pallete", "rb"))

width, height = (1024, 768)
width_ratio = float(opt.image_size) / width
height_ratio = float(opt.image_size) / height

In [12]:
# Sequence buffers
buffer_images = []
# load test clips
for iter, batch in enumerate(test_loader):
    image, info = batch

    # sort label info on fullrect
    image, label, behavior_label, obj_label, face_label, emo_label, frame_id = SortFullRect(
        image, info, is_train=False)

    try :
        image = torch.cat(image,0).cuda(device)
    except:
        continue

    # -----------------(2) inference -------------------------
    # emotion
    if np.array(face_label).size > 0 and False:
        face_label = [fl for fl in face_label if len(fl) > 0]
        emo_label = [el for el in emo_label if len(el) > 0]
        image_c = image.permute(0,2,3,1).cpu()
        face_crops, emo_gt = crop_face_emotion(image_c, face_label, emo_label, opt)
        face_crops, emo_gt = face_crops.cuda(device).contiguous(), emo_gt.cuda(device)
        emo_logits = model_emo(face_crops)
        num_img, num_face = np.array(face_label).shape[0:2]
        emo_logits = emo_logits.view(num_img, num_face, 7)
    
    for idx, frame in enumerate(frame_id):

        # ---------------(3) mkdir for evaluations----------------------
        f_info = frame[0].split('/')
        save_dir = './results/drama-graph/{}/{}/{}/'.format(
            f_info[4], f_info[5], f_info[6])

        f_file = f_info[7]
        mAP_file = "{}_{}_{}_{}".format(f_info[4],
                                        f_info[5],
                                        f_info[6],
                                        f_info[7].replace("jpg", "txt"))
        if opt.display:
            print("frame.__len__{}, mAP_file:{}".format(len(frame_id), mAP_file))
            
        # --------------(5) visualization of inferences ----------
        # out of try : pdb.set_trace = lambda : None
        try:
            # for some empty video clips
            img = image[idx]
            # ToTensor function normalizes image pixel values into [0,1]
            np_img = img.cpu().numpy()
            np_img = np.transpose(np_img,(1,2,0)) * 255
            output_image = cv2.cvtColor(np_img,cv2.COLOR_RGB2BGR)
            output_image = cv2.resize(output_image, (width, height))

            if len(predictions_p) != 0 :
                prediction = predictions[idx]
                
                if False:
                    b_logit = b_logits[idx]

                # person and behavior
                num_preds = len(prediction)
                for jdx, pred in enumerate(prediction):
                    # emotion
                    if False:
                        fl = face_label[idx][jdx]
                        face_x0, face_y0 = int(fl[0]/width_ratio), int(fl[1]/height_ratio)
                        face_x1, face_y1 = int(fl[2]/width_ratio), int(fl[3]/height_ratio)
                        emo_ij = F.softmax(emo_logits[idx,jdx,:], dim=0).argmax().detach().cpu().numpy()
                        emo_txt = EmoCLS[emo_ij]
                        cv2.rectangle(output_image, (face_x0,face_y0),
                                      (face_x1,face_y1), (255,255,0), 1)
                        cv2.putText(output_image, emo_txt, (face_x0, face_y0-5),
                                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,0), 1,
                                    cv2.LINE_AA)
                    
                    if opt.display:
                        print("detected {}".format(save_dir + "{}".format(f_file)))
                else:
                    if opt.display:
                        print("non-detected {}".format(
                        save_dir + "{}".format(f_file)))

            # save output image  
            cv2.imwrite(save_dir + "{}".format(f_file), output_image)
            # save images
            plt_output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
            plt.imshow(plt_output_image.astype('uint8'))
            plt.show()
            plt.close()
        except:
            continue

/usr/local/envs/vtt_env/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002438.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002446.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002454.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002462.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002470.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002478.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002486.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002494.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002502.txt
frame.__len__10, mAP_file:AnotherMissOh07_001_0035_IMAGE_0000002510.txt
frame.__len__10, mAP_file:AnotherMissOh07_002_0036_IMAGE_0000002568.txt
frame.__len__10, mAP_file:AnotherMissOh07_002_0036_IMAGE_0000002576.txt
frame.__len__10, mAP_file:AnotherMissOh07_002_0036_IMAGE_0000002584.txt
frame.__len__10, mAP_file:AnotherMissOh07_002_0036_IMAGE_0000002

frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006235.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006243.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006251.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006259.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006267.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006275.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006283.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006291.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006299.txt
frame.__len__10, mAP_file:AnotherMissOh07_004_0056_IMAGE_0000006307.txt
frame.__len__2, mAP_file:AnotherMissOh07_004_0057_IMAGE_0000006314.txt
frame.__len__2, mAP_file:AnotherMissOh07_004_0057_IMAGE_0000006322.txt
frame.__len__2, mAP_file:AnotherMissOh07_004_0058_IMAGE_0000006335.txt
frame.__len__2, mAP_file:AnotherMissOh07_004_0058_IMAGE_0000006343.

KeyboardInterrupt: 

In [ ]:
        # --------------(4) ground truth ---------------------------------
        # save person ground truth
        gt_person_cnt = 0
        if len(label) > idx :
            # person
            for det in label[idx]:
                cls = PersonCLS[int(det[4])]
                xmin = str(max(det[0] / width_ratio, 0))
                ymin = str(max(det[1] / height_ratio, 0))
                xmax = str(min((det[2]) / width_ratio, width))
                ymax = str(min((det[3]) / height_ratio, height))
                cat_det = '%s %s %s %s %s\n' % (cls, xmin, ymin, xmax, ymax)
                if opt.display:
                    print("person_gt:{}".format(cat_det))
                gt_person_cnt += 1
                
            # behavior
            for j, det in enumerate(label[idx]):
                cls = PBeHavCLS[int(behavior_label[idx][j])].replace(' ', '_')
                if cls == 'none':
                    continue

                cls = cls.replace('/', '_')
                xmin = str(max(det[0] / width_ratio, 0))
                ymin = str(max(det[1] / height_ratio, 0))
                xmax = str(min((det[2]) / width_ratio, width))
                ymax = str(min((det[3]) / height_ratio, height))
                cat_det = '%s %s %s %s %s\n' % (cls, xmin, ymin, xmax, ymax)
                if opt.display:
                    print("behavior_gt:{}".format(cat_det))
        # object
        gt_object_cnt = 0
        if len(obj_label) > idx :
            for det in obj_label[idx]:
                cls = ObjectCLS[int(det[4])]
                xmin = str(max(det[0] / width_ratio, 0))
                ymin = str(max(det[1] / height_ratio, 0))
                xmax = str(min((det[2]) / width_ratio, width))
                ymax = str(min((det[3]) / height_ratio, height))
                cat_det = '%s %s %s %s %s\n' % (cls, xmin, ymin, xmax, ymax)
                if opt.display:
                    print("object_gt:{}".format(cat_det))
                gt_object_cnt += 1
                
        # relation
        gt_relation_cnt = 0
        if len(obj_label) > idx:
            for det in obj_label[idx]:
                cls = P2ORelCLS[int(det[5])]
                xmin = str(max(det[0] / width_ratio, 0))
                ymin = str(max(det[1] / height_ratio, 0))
                xmax = str(min((det[2]) / width_ratio, width))
                ymax = str(min((det[3]) / height_ratio, height))
                cat_det = '%s %s %s %s %s\n' % (cls, xmin, ymin, xmax, ymax)
                if opt.display:
                    print("relation_gt:{}".format(cat_det))
                gt_relation_cnt += 1

        # place
        if len(preds_place_txt) > idx:
            if opt.display:
                print("place_gt:{}".format(target_place_txt[idx]))
        # face
        gt_face_cnt = 0
        if len(face_label) > idx:
            for det in face_label[idx]:
                cls = PersonCLS[int(det[4])]
                xmin = str(max(det[0] / width_ratio, 0))
                ymin = str(max(det[1] / height_ratio, 0))
                xmax = str(min((det[2]) / width_ratio, width))
                ymax = str(min((det[3]) / height_ratio, height))
                cat_det = '%s %s %s %s %s\n' % (cls, xmin, ymin, xmax, ymax)
                print("face_gt:{}".format(cat_det))
                gt_face_cnt += 1